# Modelling v1

modelling use the latest exploration

# Goal

Get the model from the prepared data. Without oversampling. 

# Plan

Checklist what will be done on the notebook :

    [*] Get Data
    [ ] Modelling



In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic
import time

from scipy.stats import uniform,randint
import warnings
warnings.filterwarnings("ignore")

In [2]:
with open("../../config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

PATH_RAW = config['path']['raw']
PATH_INTERIM = config['path']['interim']
PATH_PROCESSED = config['path']['processed']

if os.name == 'posix' :
    PATH_MODEL = config['path']['artifact']+os.getcwd().split('/')[-1]+'/'
else:
    PATH_MODEL = config['path']['artifact']+os.getcwd().split('\\')[-1]+'/'

PATH_UDF = config['path']['udf']

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

## Get Data

In [4]:
raw_data = pd.read_csv(PATH_PROCESSED+'prep_iter2.csv')
col_cat_load = joblib.load(PATH_MODEL+'cat_names.pkl')
col_num_load = joblib.load(PATH_MODEL+'num_names.pkl')

In [5]:
X_tr = raw_data[raw_data.is_train == 1][col_num_load+col_cat_load]
Y_tr = raw_data[raw_data.is_train == 1]['target'].values

X_val = raw_data[raw_data.is_train == 0][col_num_load+col_cat_load]
Y_val = raw_data[raw_data.is_train == 0]['target'].values

In [6]:
X_tr.shape

(32665, 26)

In [7]:
X_val.shape

(5765, 26)

In [8]:
Y_val.shape

(5765,)

## Modelling

In [9]:
proportion_cls = pd.Series(Y_tr).value_counts(normalize = True)
scale_pos_w = np.ceil(proportion_cls[0]/proportion_cls[1])
cw = {0:1, 1:scale_pos_w}
cw
# scale_pos_weight for xgboost

{0: 1, 1: 8.0}

In [10]:
proportion_cls

0    0.884525
1    0.115475
dtype: float64

In [11]:
all_model = {'logreg' : LogisticRegression(class_weight=cw)
            ,'svm' : SVC(class_weight=cw, probability=True)
            ,'rf': RandomForestClassifier(class_weight=cw, random_state=234)
            ,'nb':GaussianNB(priors=[0.885,0.115])}

In [12]:
param_options = {'logreg' : dict(C=uniform(loc=0, scale=4)
                                 ,penalty=['l2', 'l1']
                                 ,solver=['liblinear', 'sag', 'saga'])
                 ,'svm' : dict(C=uniform(loc=0, scale=4)
                              ,kernel=['linear', 'poly', 'rbf', 'sigmoid']
                              ,degree= randint(2, 6)
                              ,coef0=uniform(loc=0, scale=2))  
                 ,'rf' : dict(n_estimators=randint(300, 600)
                             ,max_depth=randint(15, 35)
                             ,min_samples_split=randint(3, 10)
                             ,min_samples_leaf=randint(3, 10)
                             ,min_weight_fraction_leaf=uniform(loc=0, scale=0.3)
                             ,min_impurity_decrease=uniform(loc=0, scale=0.7))
                 ,'nb' : dict(var_smoothing = uniform(loc=1e-8, scale=1e-2))
                }

 # min_weight_fraction_leaf must in [0, 0.5]

In [ ]:
## scoring
## https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
best_clf = {}
best_scr = {}
sla_model = {}
for mdl in all_model.keys():
    L1 = time.time()
    clf = all_model[mdl]
    parm = param_options[mdl]
    RS = RandomizedSearchCV(clf, parm, n_iter = 80, cv = 5, scoring = 'roc_auc',n_jobs = 10)
    
    RS.fit(X_tr,Y_tr)
    
    best_clf[mdl] = RS.best_estimator_
    best_scr[mdl] = RS.best_score_
    diff_time = round((time.time()-L1)/60, 2)
    
    joblib.dump(best_clf[mdl],PATH_MODEL+mdl+'.pkl')
    
    print(mdl, 'is done in ',diff_time, ' minutes')
    sla_model[mdl] = diff_time

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number 

logreg is done in  4.99  minutes


In [ ]:
sla_model

In [ ]:
best_scr

In [ ]:
best_clf

## Evaluation

In [ ]:
Y_prob_pred = best_clf['svm'].predict_proba(X_val)[:,1]
Y_val